Laboration för kursen DD1334
Använder SQL-lite för att kommunicera med en databas som sparar data från vårt projekt i kartgenerering

Först måste vi hämta hem map klasser från github och installera dependencies

In [35]:
! pip install depend #fixa sen
%pip install ipywidgets
! git clone git@github.com:poskusen/dd1334.git

Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#fixa': Expected package name at the start of dependency specifier
    #fixa
    ^


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Cloning into 'dd1334'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


Vi börjar med att generera en karta som ska sparas.

In [43]:
from Classes.map import Map
import ipywidgets as widgets
from IPython.display import display, clear_output
from draw_PNG import draw_map
ny_karta = Map(1, 50, 0, 0 ,100, realm_name= 'Generated_Map') # Skapa en karta med bara byar

draw_map(ny_karta)
output = widgets.Output()
with output: # Visa upp kartan
    image = open("test_image.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)


Output()

Sen connectar vi till databasen och skapar ett nytt table för kartor, continenter och allt content på kontinenterna.

Här uppstår ett stort problem som vi undviker genom att använda pickle.
För att spara vår representation av kontinenten så måste vi spara en lista med punkter. Detta blir väldigt tricky då SQL databaser är baserade på att man sparar 1 värde och inte en lista med värden. Något vi undviker när vi bara serialiserar hela objektet. 
En lösning hade varit skapa ett table per lista och därmed en per kontinent, men det är väldigt bökigt och förstör hela poängen med databasen. Och eftersom vi aldrig vill ha enbart en eller flera punkter från kontinent-listan finns det ingen poäng med att skapa ett helt nytt table, då vi aldrig hämtar enskilda värden från listan. Därför kan man se listan snarare som ett objekt, än en lista. Därför serialiserar vi listan och sparar den sen, allt annat hade varit galenskap.

För att ändå visa att det går gör vi det för städerna.
Vi sparar även bara städerna för floder och berg hade inneburit dubbelarbete, och visar egentligen ingenting.

Den enda fördelen med att köra sql är att det går att hämta kontinenter med rätt 'size' från databasen, dvs att jämföra storleken på serilizationen av points_list.

In [45]:
import sqlite3, csv

conn = sqlite3.connect('database.db')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = OFF;')

taboles = ['maps', 'continents', 'villages']
for table in taboles:
    cursor.execute(f"DROP TABLE IF EXISTS {table}") #Reset or drop all before filled out tables


cursor.execute('PRAGMA foreign_keys = ON;')

cursor.execute('''CREATE TABLE maps(
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               map_size_x INT DEFAULT 1000,
               map_size_y INT DEFAULT 1000,
               name VARCHAR(255) NOT NULL,
               amount_continents INTEGER NOT NULL
               );''') # Skapa ett table för kartor

cursor.execute('''CREATE TABLE continents(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                points_serialised VARCHAR(65535),
                id_map INTEGER NOT NULL,
                FOREIGN KEY (id_map) REFERENCES maps(id)
               );''') # Skapa ett table för kontintenter


cursor.execute('''CREATE TABLE villages(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                x INT NOT NULL,
                y INT NOT NULL,
                id_cont INTEGER NOT NULL,
                FOREIGN KEY (id_cont) REFERENCES continents(id)
               );''') # Skapa ett table för byar

conn.commit()

Stäng databas om det behövs

In [5]:
conn.close()

Skapa en enkel serialiserings algoritm som kan spara listor och en som kan återskapa innehållet och göra det till en lista

In [46]:
def stringify(continent_list):
    ser = ''
    for point in continent_list:
        str_point = str(point)
        str_point = str_point.strip('()')
        str_point = str_point.replace(' ','')
        ser +=  str_point + ','
    return ser

    

def unstringify(ser_cont):
    continent_list = []
    lista_nummer = ser_cont.split(',')  # Split the serialized string by commas
    print(lista_nummer)  # For debugging, you can remove this in production

    # Iterate through pairs of numbers (assuming each pair represents x, y coordinates)
    for i in range(0, len(lista_nummer) - 1, 2):
        # Convert the string representation of the numbers to floats
        continent_list.append((float(lista_nummer[i]), float(lista_nummer[i + 1])))

    return continent_list


lista = [(5,5), (6,6), (7,7)] # Testa funktionerna
string_lista = stringify(lista)
print(string_lista)
print(unstringify(string_lista))


5,5,6,6,7,7,
['5', '5', '6', '6', '7', '7', '']
[(5.0, 5.0), (6.0, 6.0), (7.0, 7.0)]


Skapa funktion som extraherar kontinenter, karta och byar och serialisera kontinent_listorna samt lägger till dem och kartan i databasen

In [47]:
def save(map, cursor):
    amount_continents = map.get_continent_amount()
    name_map = map.get_realm_name()
    (x_size, y_size) = map.get_mapsize()
    cursor.execute('''INSERT INTO maps (map_size_x, map_size_y, name, amount_continents)
                    VALUES(?,?,?,?);
                   ''', (x_size, y_size, name_map, amount_continents))
    continents = map.get_continents()
    map_id = cursor.lastrowid
    for cont in continents:
        points_list = cont.get_point_list()
        ser_points = stringify(points_list)
        cursor.execute('''INSERT INTO continents (points_serialised, id_map)
                    VALUES(?,?);
                    ''', (ser_points, map_id))
        villages = cont.get_villages().get_cities_list()
        cont_id = cursor.lastrowid
        for village in villages:
            cursor.execute('''INSERT INTO villages (x, y, id_cont)
                        VALUES(?,?,?);
                           ''', (village[0], village[1], cont_id))

Testa att spara ner en karta

In [48]:
save(ny_karta, cursor)
cursor = conn.execute("SELECT * FROM CONTINENTS")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.commit() # Commita till databasen

(1, '466.0699492302033,177.1385008302006,457.24088721972447,221.645110614952,441.76455257529193,242.30447400803763,417.23243954079487,253.2041376375584,416.31603585220006,253.12862935349807,399.0921021772628,238.49496904094616,379.5086212987047,211.0862079135789,375.94183259256823,169.6647597095669,362.26754937854815,138.19109543563923,371.30290314963514,96.12681147025933,404.8259301923955,66.35958816368918,433.5424984815101,70.0030328482622,478.1614805687718,89.80093824773297,493.3114869151789,104.62041743115503,498.7592638966818,112.28435781736277,497.57618667464294,138.08137220323948,498.6053618656588,151.84335466979553,498.6283260428279,169.40468392088388,491.51237362687607,192.35482192004162,489.23594539839667,199.23694054988817,480.7749103282663,207.17190605504817,456.30458259604364,220.29902067807342,466.0699492302033,177.1385008302006,', 1)


Skapa en funktion som hämtar upp en karta och återskapar kartan

In [49]:
def load(karta_id, cursor): 
    cursor.execute('''SELECT * FROM maps
                JOIN continents ON maps.id = continents.id_map
                JOIN villages ON continents.id = villages.id_cont
                WHERE maps.id = ?
                ;
                ''', (karta_id, )) #Hämta alla kartan med karta_id
    
    data = cursor.fetchall()
    for data in data:
        print(data)
        
    amount_continents = data[4]
    list_continents = []
    for i in range(0, amount_continents): #Hämta kontinenterna
        list_continents.append(unstringify(data[6 + i])) #Avserilisera kontinenterna
    for i in range(6 + amount_continents, len(data)): #Hämta kontinenter
        pass
    
    #return Map(amount_continents, 100, 0, 0, 50, (data[1], data[2]), data[3], create_new=False, ) #Återskapa kartan

laddad_karta = load(1, cursor)




(1, 1000, 1000, 'Generated_Map', 1, 1, '466.0699492302033,177.1385008302006,457.24088721972447,221.645110614952,441.76455257529193,242.30447400803763,417.23243954079487,253.2041376375584,416.31603585220006,253.12862935349807,399.0921021772628,238.49496904094616,379.5086212987047,211.0862079135789,375.94183259256823,169.6647597095669,362.26754937854815,138.19109543563923,371.30290314963514,96.12681147025933,404.8259301923955,66.35958816368918,433.5424984815101,70.0030328482622,478.1614805687718,89.80093824773297,493.3114869151789,104.62041743115503,498.7592638966818,112.28435781736277,497.57618667464294,138.08137220323948,498.6053618656588,151.84335466979553,498.6283260428279,169.40468392088388,491.51237362687607,192.35482192004162,489.23594539839667,199.23694054988817,480.7749103282663,207.17190605504817,456.30458259604364,220.29902067807342,466.0699492302033,177.1385008302006,', 1, 1, 456.79997652707027, 163.43193527878626, 1)
(1, 1000, 1000, 'Generated_Map', 1, 1, '466.0699492302033,

In [85]:
from Classes.continent import Continent
from Classes.map import Map

#Olle load
def load(karta_id, cursor): 
    continent_cords_list = [] #Every position is a list with all the continents positions
    villages_lists = []

    cursor.execute('''UPDATE maps 
                  SET name = 'loaded_map' 
                  WHERE id = ?;''', (karta_id,))
    conn.commit() 

    cursor.execute('''SELECT COUNT(*) 
                      FROM continents 
                      WHERE id_map = ?;''', (karta_id,))
    number_of_continents = cursor.fetchone()[0]  # Count of continents
    
    cursor.execute('''SELECT * 
                  FROM continents 
                  WHERE id_map = ?;''', (karta_id,))#Get all continents
    
    continents = cursor.fetchall()

    for row in continents:
        continent_cords_list.append(unstringify(row[1]))  #Now we have all continent cordinates 

    for continent_id in range (1, number_of_continents+1):
        temp_list = []

        cursor.execute('''SELECT * 
                      FROM villages 
                      WHERE id_cont = ?;''', (continent_id,))
        villages = cursor.fetchall()
        for rows in villages:
            x = rows[1]
            y = rows[2]
            xy = (x,y)
            temp_list.append(xy)
        villages_lists.append(temp_list)

    #Now make continent list containing actual continent instances
    continent_list = []
    for i in range (number_of_continents):
        temp_cont = Continent((1000,1000), 0, 0, 0 , 0, point_list = continent_cords_list[i], content = [None, villages_lists[i], None, None], LoadMapp=True)
        continent_list.append(temp_cont)

    #Select info about the map
    cursor.execute('''SELECT * 
                      FROM maps 
                      WHERE id = ?;''', (karta_id,))
    map_info = cursor.fetchone()  # Get the map information 
    
    loaded_map = Map(number_of_continents, 2, 2, 2, 2,  create_new=False, realm_name="Loaded Map")
    loaded_map.continent_list = continent_list

    return loaded_map


load(1, cursor)


['466.0699492302033', '177.1385008302006', '457.24088721972447', '221.645110614952', '441.76455257529193', '242.30447400803763', '417.23243954079487', '253.2041376375584', '416.31603585220006', '253.12862935349807', '399.0921021772628', '238.49496904094616', '379.5086212987047', '211.0862079135789', '375.94183259256823', '169.6647597095669', '362.26754937854815', '138.19109543563923', '371.30290314963514', '96.12681147025933', '404.8259301923955', '66.35958816368918', '433.5424984815101', '70.0030328482622', '478.1614805687718', '89.80093824773297', '493.3114869151789', '104.62041743115503', '498.7592638966818', '112.28435781736277', '497.57618667464294', '138.08137220323948', '498.6053618656588', '151.84335466979553', '498.6283260428279', '169.40468392088388', '491.51237362687607', '192.35482192004162', '489.23594539839667', '199.23694054988817', '480.7749103282663', '207.17190605504817', '456.30458259604364', '220.29902067807342', '466.0699492302033', '177.1385008302006', '']


Testa att spara och hämta en karta

In [88]:
from draw_PNG import draw_map

laddad_karta = load(1, cursor)
print(laddad_karta.mapsize)


draw_map(laddad_karta, "loaded_map.png")

output = widgets.Output()
with output: # Visa upp kartan
    image = open("loaded_map.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)

['466.0699492302033', '177.1385008302006', '457.24088721972447', '221.645110614952', '441.76455257529193', '242.30447400803763', '417.23243954079487', '253.2041376375584', '416.31603585220006', '253.12862935349807', '399.0921021772628', '238.49496904094616', '379.5086212987047', '211.0862079135789', '375.94183259256823', '169.6647597095669', '362.26754937854815', '138.19109543563923', '371.30290314963514', '96.12681147025933', '404.8259301923955', '66.35958816368918', '433.5424984815101', '70.0030328482622', '478.1614805687718', '89.80093824773297', '493.3114869151789', '104.62041743115503', '498.7592638966818', '112.28435781736277', '497.57618667464294', '138.08137220323948', '498.6053618656588', '151.84335466979553', '498.6283260428279', '169.40468392088388', '491.51237362687607', '192.35482192004162', '489.23594539839667', '199.23694054988817', '480.7749103282663', '207.17190605504817', '456.30458259604364', '220.29902067807342', '466.0699492302033', '177.1385008302006', '']
(1000, 

Output()

Testa att spara och hämta flera kartor